# **Portfolio Optimization Using Genetic Algorithm**

Load the data

In [14]:
import pandas as pd

# Load data
df = pd.read_csv('portfolio_data.csv', index_col=0, parse_dates=True)
df.head()


,Asset_1,Asset_2,Asset_3,Portfolio_Value,Market_Index
timestamp,,,,,
01-01-2020,544.883183,423.654799,645.894113,525.051834,1034.932512
02-01-2020,437.587211,891.773001,963.662760,792.579124,1026.081261
03-01-2020,383.441519,791.725038,528.894920,600.939199,1024.844657
06-01-2020,568.044561,925.596638,71.036058,564.754139,966.747919
07-01-2020,87.129300,20.218397,832.619845,290.226071,1036.577579


# **Calculate Return**

In [18]:
returns = df.pct_change().dropna()
returns.head()


,Asset_1,Asset_2,Asset_3,Portfolio_Value,Market_Index
timestamp,,,,,
02-01-2020,-0.196916,1.104952,0.491983,0.509525,-0.008552
03-01-2020,-0.123737,-0.112190,-0.451162,-0.241793,-0.001205
06-01-2020,0.481437,0.169089,-0.865690,-0.060214,-0.056688
07-01-2020,-0.846615,-0.978156,10.721087,-0.486102,0.072232
08-01-2020,7.931057,42.030717,0.175348,2.029164,-0.019868


# Define Fitness Function

In [21]:
import numpy as np

def fitness_function(weights, returns):
    portfolio_return = np.sum(returns.mean() * weights) * 252
    portfolio_stddev = np.sqrt(np.dot(weights.T, np.dot(returns.cov() * 252, weights)))
    sharpe_ratio = portfolio_return / portfolio_stddev
    return sharpe_ratio


# Initial Population

In [24]:
def initialize_population(pop_size, num_assets):
    population = []
    for _ in range(pop_size):
        weights = np.random.random(num_assets)
        weights /= np.sum(weights)
        population.append(weights)
    return np.array(population)

pop_size = 100
num_assets = len(df.columns)
population = initialize_population(pop_size, num_assets)


# Selection

In [27]:
def selection(population, fitness_scores, num_parents):
    parents = np.empty((num_parents, population.shape[1]))
    for i in range(num_parents):
        max_fitness_idx = np.where(fitness_scores == np.max(fitness_scores))
        max_fitness_idx = max_fitness_idx[0][0]
        parents[i, :] = population[max_fitness_idx, :]
        fitness_scores[max_fitness_idx] = -999999
    return parents

num_parents = 20
fitness_scores = np.array([fitness_function(weights, returns) for weights in population])
parents = selection(population, fitness_scores, num_parents)


# Crossover

In [30]:
def crossover(parents, offspring_size):
    offspring = np.empty(offspring_size)
    crossover_point = np.uint8(offspring_size[1]/2)

    for k in range(offspring_size[0]):
        parent1_idx = k % parents.shape[0]
        parent2_idx = (k + 1) % parents.shape[0]
        offspring[k, 0:crossover_point] = parents[parent1_idx, 0:crossover_point]
        offspring[k, crossover_point:] = parents[parent2_idx, crossover_point:]
    return offspring

offspring_size = (pop_size - parents.shape[0], num_assets)
offspring = crossover(parents, offspring_size)


# Mutation

In [33]:
def mutation(offspring, mutation_rate=0.01):
    for idx in range(offspring.shape[0]):
        for gene_idx in range(offspring.shape[1]):
            if np.random.rand() < mutation_rate:
                random_value = np.random.random()
                offspring[idx, gene_idx] = offspring[idx, gene_idx] + random_value
                offspring[idx, gene_idx] /= np.sum(offspring[idx, :])
    return offspring

mutated_offspring = mutation(offspring)


# Termination

In [36]:
def mutation(offspring, mutation_rate=0.01):
    for idx in range(offspring.shape[0]):
        for gene_idx in range(offspring.shape[1]):
            if np.random.rand() < mutation_rate:
                random_value = np.random.random()
                offspring[idx, gene_idx] = offspring[idx, gene_idx] + random_value
                offspring[idx, gene_idx] /= np.sum(offspring[idx, :])
    return offspring

mutated_offspring = mutation(offspring)


In [39]:
def genetic_algorithm(returns, num_generations, pop_size, num_parents, mutation_rate):
    num_assets = returns.shape[1]
    population = initialize_population(pop_size, num_assets)
    for generation in range(num_generations):
        fitness_scores = np.array([fitness_function(weights, returns) for weights in population])
        parents = selection(population, fitness_scores, num_parents)
        offspring_size = (pop_size - parents.shape[0], num_assets)
        offspring = crossover(parents, offspring_size)
        offspring = mutation(offspring, mutation_rate)
        population[0:parents.shape[0], :] = parents
        population[parents.shape[0]:, :] = offspring
        best_fitness = np.max(fitness_scores)
        print(f"Generation {generation}: Best Fitness = {best_fitness}")
    return population, fitness_scores

num_generations = 100
population, fitness_scores = genetic_algorithm(returns, num_generations, pop_size, num_parents, mutation_rate=0.01)


Generation 0: Best Fitness = 1.4762715381515883
Generation 1: Best Fitness = 2.02523213483615
Generation 2: Best Fitness = 2.447525052066359
Generation 3: Best Fitness = 2.5968127611299243
Generation 4: Best Fitness = 2.601141911178252
Generation 5: Best Fitness = 2.601323996364056
Generation 6: Best Fitness = 2.651556189371385
Generation 7: Best Fitness = 2.651556189371385
Generation 8: Best Fitness = 2.651871247334236
Generation 9: Best Fitness = 2.651871247334236
Generation 10: Best Fitness = 2.6585766530068415
Generation 11: Best Fitness = 2.6706997052344295
Generation 12: Best Fitness = 2.697812640265176
Generation 13: Best Fitness = 2.697812640265176
Generation 14: Best Fitness = 2.697812640265176
Generation 15: Best Fitness = 2.6988428788994336
Generation 16: Best Fitness = 2.6989832050307423
Generation 17: Best Fitness = 2.6989832050307423
Generation 18: Best Fitness = 2.6998114819174277
Generation 19: Best Fitness = 2.7029987211167135
Generation 20: Best Fitness = 2.7029987211

# Evaluate Best portfolio

In [42]:
best_idx = np.argmax(fitness_scores)
best_portfolio = population[best_idx, :]
best_portfolio_return = np.sum(returns.mean() * best_portfolio) * 252
best_portfolio_stddev = np.sqrt(np.dot(best_portfolio.T, np.dot(returns.cov() * 252, best_portfolio)))
best_portfolio_sharpe = best_portfolio_return / best_portfolio_stddev

print(f"Best Portfolio Weights: {best_portfolio}")
print(f"Expected Annual Return: {best_portfolio_return}")
print(f"Annual Volatility: {best_portfolio_stddev}")
print(f"Sharpe Ratio: {best_portfolio_sharpe}")


Best Portfolio Weights: [0.01644336 0.08476277 0.52198998 0.62815317 0.5433535 ]
Expected Annual Return: 492.3823055102672
Annual Volatility: 178.2089763198907
Sharpe Ratio: 2.7629489584544022
